In [ ]:
# Install the necessary package
# !pip install vanna[bigquery] google-cloud

# Import necessary libraries
import vanna
from vanna.remote import VannaDefault
import os
from google.cloud import bigquery
from google.oauth2 import service_account

# Set your API key and model name (replace these placeholders)
api_key = 'xxxxxxxxxxx'  # Replace with your actual API key
vanna_model_name = 'cfg'  # Replace with your actual model name

# Initialize Vanna API connection
vn = VannaDefault(model=vanna_model_name, api_key=api_key)

# Set up BigQuery credentials
# Define local path to the credentials file
LOCAL_CREDENTIALS_PATH = "/content/jasonlbigquery-eb26fab8b369.json"

# Set environment variable for authentication
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = LOCAL_CREDENTIALS_PATH

# Authenticate using the service account credentials
bq_credentials = service_account.Credentials.from_service_account_file(LOCAL_CREDENTIALS_PATH)
client = bigquery.Client(credentials=bq_credentials, project=bq_credentials.project_id)

# Connect to BigQuery using Vanna
vn.connect_to_bigquery(project_id=bq_credentials.project_id)

# Step 1: Retrieve the information schema (column metadata) from BigQuery for the table `monthly_ga_data`
df_information_schema = vn.run_sql("SELECT * FROM `jasonlbigquery.reports.INFORMATION_SCHEMA.COLUMNS` WHERE table_name = 'monthly_ga_data'")

# Step 2: Create a training plan using the retrieved schema
plan = vn.get_training_plan_generic(df_information_schema)

# Optional: If you like the plan, uncomment the following line to start training
# This will add the schema to Vanna's knowledge base
vn.train(plan=plan)

# Optional: Adding more training data
# Add a DDL statement for your table to teach the model about your table structure
vn.train(ddl="""CREATE TABLE IF NOT EXISTS `jasonlbigquery.reports.monthly_ga_data` (
    ga_id INT64,
    ga_date DATE,
    username STRING,
    sessions INT64,
    pageviews INT64,
    bounce_rate FLOAT64
)""")

# Add some documentation to help the model understand your business terms
vn.train(documentation="The `sessions` field represents the number of sessions in a given month, and the `bounce_rate` represents the percentage of sessions where users left without interacting further.")

# Optionally, add an SQL query that might be helpful for training
vn.train(sql="SELECT * FROM `jasonlbigquery.reports.monthly_ga_data` WHERE username = 'Julian Naidoo'")

# Step 3: Inspect your training data to ensure it's what you expect
training_data = vn.get_training_data()
print(f"\nTraining data:\n{training_data}")

# Now you can use Vanna for SQL generation with your trained data

# Function to handle user input and interact with Vanna API
def converse():
    print("Welcome to the Vanna conversational interface! Type 'exit' to quit.")

    while True:
        # User prompt to ask a question
        question = input("\nPlease enter your question: ")

        if question.lower() == 'exit':
            print("Exiting the program.")
            break

        # Add context to the question, such as the BigQuery table
        full_question = f"Use the `jasonlbigquery.reports.monthly_ga_data` table to {question}"

        # Generate SQL dynamically from the question using Vanna
        try:
            # Use Vanna to generate the SQL query
            sql = vn.ask(question=full_question)
            print(f"\nGenerated SQL:\n{sql}")

            # Run the generated SQL query against BigQuery
            query_job = client.query(sql)  # Running the query
            results = query_job.result()   # Getting the results

            # Displaying the results
            for row in results:
                print(f"Result: {row}")  # You can format this as needed

        except Exception as e:
            print(f"Error: {e}")

if __name__ == "__main__":
    converse()


Adding ddl: CREATE TABLE IF NOT EXISTS `jasonlbigquery.reports.monthly_ga_data` (
    ga_id INT64,
    ga_date DATE,
    username STRING,
    sessions INT64,
    pageviews INT64,
    bounce_rate FLOAT64
)
Adding documentation....
Question generated with sql: What are the monthly data reports for the user Julian Naidoo? 
Adding SQL...

Training data:
             id training_data_type  \
0   1045650-sql                sql   
1   3310315-doc      documentation   
2   4643451-ddl                ddl   
3   4646895-ddl                ddl   
4   4648859-ddl                ddl   
5   1045648-sql                sql   
6   3310328-doc      documentation   
7   1045651-sql                sql   
8   3310329-doc      documentation   
9   4646712-ddl                ddl   
10  1045647-sql                sql   
11  3310316-doc      documentation   
12  1045649-sql                sql   
13  3310319-doc      documentation   
14  3310310-doc      documentation   
15  3310314-doc      documentation   
16